In [120]:
import os
import json
import pandas as pd
from neo4j import GraphDatabase

In [160]:
dir = "/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-4"

In [161]:
def list_directories(path):
    # List only directories
    directories = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return directories

def list_files(directory):
    # List only files
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    return files

def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)  # Parse JSON data
    return data

def convert_dependency_format_to_list(data):
    result = []
    for dependency in data:
        if ":" in dependency['artifact']:
            converted_artifact = ":".join(dependency['artifact'].split(':')[0:2])
            result.append(converted_artifact)
    return list(set(result))        

In [163]:
from datetime import datetime
import ast

class ComparedResult:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.df = pd.read_csv(csv_file)    
        
    def get_timestamp(self, release_id):
        artifact_id = ":".join(release_id.split(":")[0:2])
        version = release_id.split(":")[-1]
        row = self.df[self.df['artifact_id'] == artifact_id]
        row = row.iloc[0]
        tags = ast.literal_eval(row['tags'])
        versions = ast.literal_eval(row['versions'])
        
        compared_result_list = []
        for entry in row["compared_result"]:
            compared_result_list.append(entry[0])
            
        for entry in versions:
            if entry["name"] == version:
                return entry["ts"]
    
        # Check in older_tags
        for entry in tags:
            if entry["name"] == version:
                iso_date = entry["date"]
                timestamp = int(datetime.strptime(iso_date, "%Y-%m-%dT%H:%M:%SZ").timestamp()) * 1000
                return timestamp
    
        # If not found
        return None

In [164]:
class Neo4jDriver:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def get_timestamp(self, release_id):
        query = """
        MATCH (r:Release {id: $release_id})
        RETURN r.timestamp
        """
        with self.driver.session() as session:
            # Execute the query and return the results
            results = session.run(query, release_id=release_id)
            return [record["r.timestamp"] for record in results][0]

In [165]:
class ProjectReport:
    artifact = ""
    dir = ""
    timestamp = ""
    introduce_unused_dependency = []
    introduce_unused_dependency_but_import = []
    unused_previously_used = []
    unused_previously_used_but_import = []
    unused_dependency = []
    use_dependency = []
    use_transitive_dependency = []
    
    def __init__(self, artifact, introduce_unused_dependency, introduce_unused_dependency_but_import, unused_previously_used, unused_previously_used_but_import, unused_dependency, unused_but_import, use_dependency, use_transitive_dependency):
        self.artifact = artifact
        self.introduce_unused_dependency = introduce_unused_dependency
        self.introduce_unused_dependency_but_import = introduce_unused_dependency_but_import
        self.unused_previously_used = unused_previously_used
        self.unused_previously_used_but_import = unused_previously_used_but_import
        self.unused_dependency = unused_dependency
        self.unused_but_import = unused_but_import
        self.use_dependency = use_dependency
        self.use_transitive_dependency = use_transitive_dependency
        
    def convert_to_df(self):
        artifact_id = ":".join(self.artifact.split(":")[0:2])
        version = self.artifact.split(":")[2]
        data = {
            "artifact": artifact_id,
            "version": version,
            "introduce_unused_dependency": len(self.introduce_unused_dependency),
            "introduce_unused_dependency_but_import": len(self.introduce_unused_dependency_but_import),
            "introduce_unused_previously_used": len(self.unused_previously_used),
            "introduce_unused_but_import_previously_used": len(self.unused_previously_used_but_import),
            "unused_dependency": len(self.unused_dependency),
            "unused_dependency_but_import": len(self.unused_but_import),
            "use_dependency": len(self.use_dependency),
            "use_transitive_dependency": len(self.use_transitive_dependency),
        }
        return pd.DataFrame(data, index=[0])
        

In [166]:
list_directories(dir)

['/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-4/nl.hsac:hsac-fitnesse-email',
 '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-4/io.github.stefanofornari:vfs2nio',
 '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-4/com.squareup.pagerduty:pagerduty-incidents',
 '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-4/uk.co.rgordon:oj-ssh',
 '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-4/de.vandermeer:ascii-utf-themes',
 '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-4/io.quarkus.qlue:qlue',
 '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-4/com.forgerock.openbanking:o

In [167]:
success_project = pd.read_csv(dir + "/success_project.csv")
# success_project = pd.read_csv("temp.csv")
success_path_list = []
for index, row in success_project.iterrows():
    success_path_list.append(row["artifact_id"])

success_path_list

['org.terrakube.terraform:terraform-spring-boot-autoconfigure',
 'nl.axians:axians-opentelemetry-spring-boot-starter',
 'com.geetask:chunked-uploader',
 'com.geetask:chunked-uploader',
 'net.distilledcode:filevault-oak-reindex-hook',
 'org.java-websocket:Java-WebSocket',
 'com.github.doobo:openpdf-ivs',
 'com.github.robtimus:obfuscation-json',
 'com.s24:redjob',
 'org.sourcelab:kafka-webview-ui',
 'io.github.hiskrtapps:apocalypse-dao-springjdbc',
 'net.cactusthorn.config:jasypt',
 'nl.hsac:hsac-fitnesse-email',
 'cz.alry:plantuml-maven-plugin',
 'com.charlyghislain.authenticator:authenticator-application-api',
 'media.pepperpot.tca:tomcat-jwt-authenticator',
 'ch.rasc:wampspring',
 'com.github.bartlomiej-gora:RPNLibrary',
 'be.idamediafoundry.sofa:mapper-api',
 'se.sawano.eureka:legacy-registrar',
 'io.split.integrations.azure:azure-spring-cloud-feature-management',
 'io.probedock.rt.client:probedock-rt-itf',
 'org.ibboost.orqa:orqa-maven-plugin',
 'com.github.usc:wechat-mp-sdk',
 'org

In [168]:
len(success_path_list)

419

In [169]:
success_project

,artifact_id,repo_path
0,org.terrakube.terraform:terraform-spring-boot-...,/Users/nabhansuwanachote/Desktop/research/msr-...
1,nl.axians:axians-opentelemetry-spring-boot-sta...,/Users/nabhansuwanachote/Desktop/research/msr-...
2,com.geetask:chunked-uploader,/Users/nabhansuwanachote/Desktop/research/msr-...
3,com.geetask:chunked-uploader,/Users/nabhansuwanachote/Desktop/research/msr-...
4,net.distilledcode:filevault-oak-reindex-hook,/Users/nabhansuwanachote/Desktop/research/msr-...
...,...,...
414,com.github.livesense:org.liveSense.service.sec...,/Users/nabhansuwanachote/Desktop/research/msr-...
415,cz.etnetera:reesmo-adapter-junit,/Users/nabhansuwanachote/Desktop/research/msr-...
416,org.jboss.pnc:pnc-api,/Users/nabhansuwanachote/Desktop/research/msr-...
417,jp.igapyon.blanco.apex.formatter.plugin:blanco...,/Users/nabhansuwanachote/Desktop/research/msr-...


In [381]:
# success_project.to_csv("temp.csv")

In [181]:
# driver = Neo4jDriver("bolt://localhost:7687", "neo4j", "12345678")
filtered_directories = []
compare_result = ComparedResult("data_with_date/success-compared-result-2.csv")

for directory in list_directories(dir):
    if (directory.split("/")[-1] in success_path_list):
        filtered_directories.append(directory)

project_result = {}
count = 0
for project_dir in filtered_directories:
    list_of_files = list_files(project_dir)
    data_with_timestamp = []
    project_report = []
    project_id = ":".join(read_json_file(os.path.join(project_dir, list_of_files[0]))["projectArtifact"].split(":")[0:2])
    for file in list_of_files:
        data = read_json_file(os.path.join(project_dir, file))
        artifact_id = data["projectArtifact"]
        timestamp = compare_result.get_timestamp(artifact_id)
        data_with_timestamp.append((data, timestamp))
    data_with_timestamp.sort(key=lambda x: x[1])
    
    for index, data in enumerate(data_with_timestamp):
        current_data = data[0]
        artifact_id = current_data["projectArtifact"]
        current_unused_dependency = current_data["projectUnusedDependencies"]
        current_use_dependency = current_data["projectUseDependencies"]
        current_use_transitive_dependency = current_data["projectUseTransitiveDependencies"]
        converted_unused_dependency = list(set(convert_dependency_format_to_list(current_unused_dependency)))
        converted_use_dependency = list(set(convert_dependency_format_to_list(current_use_dependency)))
        converted_use_transitive_dependency = list(set(convert_dependency_format_to_list(current_use_transitive_dependency)))
        file_import_reports = current_data["fileImportReports"]
        
        unused_import_artifact = []
        for import_report in file_import_reports:
            unused_import_report = import_report["unusedImportReport"]
            for unused_import in unused_import_report:
                artifact = unused_import["artifact"]
                result = convert_dependency_format_to_list([artifact])
                if len(result) > 0:
                    unused_import_artifact.append(result[0])
                    
        introduce_unused_but_import = []
        unused_but_import_previously_used = []
        
        # Count unused but imported
        unused_but_imported = []
        for unused_dependency in converted_unused_dependency:
            if unused_dependency in unused_import_artifact:
                unused_but_imported.append(unused_dependency)
                
        
        # Count introduce unused dependency
        introduce_unused = []
        unused_previously_used = []
        if index == 0:
            # First version
            for unused_dep in converted_unused_dependency:
                introduce_unused.append(unused_dep)
                if unused_dep in unused_import_artifact:
                    introduce_unused_but_import.append(unused_dep)
        else:
            previous_data = data_with_timestamp[index - 1][0]
            previous_unused_dependency = previous_data["projectUnusedDependencies"]
            previous_use_dependency = previous_data["projectUseDependencies"]
            previous_use_transitive_dependency = previous_data["projectUseTransitiveDependencies"]
            all_previous_dependency = previous_unused_dependency + previous_use_dependency + previous_use_transitive_dependency
            all_previous_use_dependency = previous_use_dependency + previous_use_transitive_dependency
            converted_all_previous_dependency = list(set(convert_dependency_format_to_list(all_previous_dependency)))
            converted_all_previous_use_dependency = list(set(convert_dependency_format_to_list(all_previous_use_dependency)))
            
            for unused_dep in converted_unused_dependency:
                if unused_dep not in converted_all_previous_dependency:
                    introduce_unused.append(unused_dep)
                    if unused_dep in unused_import_artifact:
                        introduce_unused_but_import.append(unused_dep)
                elif unused_dep in converted_all_previous_use_dependency:
                    unused_previously_used.append(unused_dep)
                    if unused_dep in unused_import_artifact:
                        unused_but_import_previously_used.append(unused_dep)
        
        report = ProjectReport(artifact_id, introduce_unused, introduce_unused_but_import, unused_previously_used, unused_but_import_previously_used, converted_unused_dependency, unused_but_imported, converted_use_dependency, converted_use_transitive_dependency)
        project_report.append(report)
    project_result[project_id] = project_report

print(len(project_result))

418


In [182]:
df_list = []

for k, v in project_result.items():
    for project in v:
        df_list.append(project.convert_to_df())

concat_df = pd.concat(df_list, axis=0, ignore_index=True)
concat_df["all_introduce"] = concat_df["introduce_unused_dependency"] + concat_df["introduce_unused_previously_used"]
concat_df

,artifact,version,introduce_unused_dependency,introduce_unused_dependency_but_import,introduce_unused_previously_used,introduce_unused_but_import_previously_used,unused_dependency,unused_dependency_but_import,use_dependency,use_transitive_dependency,all_introduce
0,nl.hsac:hsac-fitnesse-email,0.0.1,1,0,0,0,1,0,2,0,1
1,nl.hsac:hsac-fitnesse-email,0.1.0,0,0,0,0,1,0,2,0,0
2,nl.hsac:hsac-fitnesse-email,0.2.0,0,0,0,0,1,0,2,0,0
3,nl.hsac:hsac-fitnesse-email,0.3.0,0,0,0,0,1,0,2,0,0
4,nl.hsac:hsac-fitnesse-email,0.4.0,0,0,0,0,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...
3016,dev.mccue:guava,0.0.5,0,0,0,0,13,0,0,0,0
3017,dev.mccue:guava,33.1.0,0,0,0,0,13,0,0,0,0
3018,dev.mccue:guava,33.2.0,0,0,0,0,13,0,0,0,0
3019,org.openstates:openstates-client,1.0.0,0,0,0,0,0,0,1,0,0


In [45]:
concat_df_result = concat_df.groupby("artifact").agg(
    introduce_unused_dependency_sum=pd.NamedAgg(column="introduce_unused_dependency", aggfunc="sum"),
    introduce_unused_dependency_but_import_sum=pd.NamedAgg(column="introduce_unused_dependency_but_import", aggfunc="sum"),
    introduce_unused_previously_use = pd.NamedAgg(column="introduce_unused_previously_used", aggfunc="sum"),
    introduce_unused_but_import_previously_used = pd.NamedAgg(column="introduce_unused_but_import_previously_used", aggfunc="sum"),
    total_unused_dependency = pd.NamedAgg(column="unused_dependency", aggfunc="sum"),
    total_unused_dependency_but_import = pd.NamedAgg(column="unused_dependency_but_import", aggfunc="sum"),
    total_use_transitive_dependency = pd.NamedAgg(column="use_transitive_dependency", aggfunc="sum"),
    median_unused_dependency = pd.NamedAgg(column="unused_dependency", aggfunc="median"),
    average_unused_dependency=pd.NamedAgg(column="unused_dependency", aggfunc="mean"),
    average_unused_dependency_but_import = pd.NamedAgg(column="unused_dependency_but_import", aggfunc="mean"),
    average_use_transitive_dependency=pd.NamedAgg(column="use_transitive_dependency", aggfunc="mean"),
    version_count=pd.NamedAgg(column="version", aggfunc="count"),
)

concat_df_result["version_count"] = concat_df_result["version_count"] + 1

# Display the result
# concat_df_result.to_csv("data/temp.csv")

In [183]:
statistics = {
    "min": concat_df.min(numeric_only=True),
    "mean": concat_df.mean(numeric_only=True),
    "median": concat_df.median(numeric_only=True),
    "max": concat_df.max(numeric_only=True),
    "total": concat_df.sum(numeric_only=True),
}

stats_df = pd.DataFrame(statistics)

# Display the statistics table
stats_df

,min,mean,median,max,total
introduce_unused_dependency,0,0.167825,0.0,13,507
introduce_unused_dependency_but_import,0,0.001324,0.0,1,4
introduce_unused_previously_used,0,0.009930,0.0,2,30
introduce_unused_but_import_previously_used,0,0.000662,0.0,1,2
unused_dependency,0,1.016220,0.0,13,3070
unused_dependency_but_import,0,0.006951,0.0,1,21
use_dependency,0,3.679907,2.0,48,11117
use_transitive_dependency,0,0.508441,0.0,10,1536
all_introduce,0,0.177756,0.0,13,537


In [184]:
group_df = concat_df.groupby("artifact").sum()
print(len(group_df[group_df["all_introduce"] > 0])/len(group_df))

0.5167464114832536


In [185]:
len(concat_df[concat_df["all_introduce"] > 0])/len(concat_df)

0.09268454154253558

In [186]:
print(len(group_df))

418


In [140]:
rq2_project_result = {}
for k, v in project_result.items():
    if k in concat_df["artifact"].tolist():
        rq2_project_result[k] = v

In [141]:
rq2_project_result

{'nl.hsac:hsac-fitnesse-email': [<__main__.ProjectReport at 0x1419e7a70>,
 'io.github.stefanofornari:vfs2nio': [<__main__.ProjectReport at 0x1419e76e0>,
 'com.squareup.pagerduty:pagerduty-incidents': [<__main__.ProjectReport at 0x1419e6c00>,
 'uk.co.rgordon:oj-ssh': [<__main__.ProjectReport at 0x1419e79b0>],
 'de.vandermeer:ascii-utf-themes': [<__main__.ProjectReport at 0x1419e64b0>],
 'io.quarkus.qlue:qlue': [<__main__.ProjectReport at 0x1430e6270>,
 'com.forgerock.openbanking:openbanking-sdk': [<__main__.ProjectReport at 0x14195a0c0>,
 'de.jpdigital:hibernate6-ddl-maven-plugin-core': [<__main__.ProjectReport at 0x1430dfb60>,
 'com.github.kasiprasad.contextspec:contextspec': [<__main__.ProjectReport at 0x1419e56a0>],
 'io.github.eduramiba:webcam-capture-driver-native': [<__main__.ProjectReport at 0x14397dfd0>],
 'org.mapdb:mapdb-hz-offheap': [<__main__.ProjectReport at 0x1430dfef0>,
 'com.pastdev:jsch-nio': [<__main__.ProjectReport at 0x1430ded20>,
 'tech.energyit:fast-java-statsd-cli

## RQ2

In [142]:
class ArtifactExistReport:
    def __init__(self, project_id, artifact_id, from_ts, to_ts, from_version, to_version, version_different, last_state):
        self.project_id = project_id
        self.artifact_id = artifact_id
        self.from_ts = from_ts
        self.to_ts = to_ts
        self.last_state = last_state
        self.from_version = from_version
        self.to_version = to_version
        self.version_different = version_different
        
    def convert_to_df(self):
        duration = None
        if self.to_ts != None:
            duration = self.to_ts - self.from_ts
        data = {
            "project_id": self.project_id,
            "artifact": self.artifact_id,
            "from_ts": self.from_ts,
            "to_ts": self.to_ts,
            "last_state": self.last_state,
            "from_version": self.from_version,
            "to_version": self.to_version,
            "version_different": self.version_different,
            "duration": duration,
        }
        return pd.DataFrame(data, index=[0])
    

In [143]:
# driver = Neo4jDriver("bolt://localhost:7687", "neo4j", "12345678")


def find_evolution(current_index, last_index, project_result, unused_artifact_id, project_id):
    from_ts = project_result[current_index].timestamp
    from_version = project_result[current_index].artifact.split(":")[2]
    for i in range(current_index + 1, last_index + 1):
        current_project_result = project_result[i]
        if unused_artifact_id in current_project_result.unused_dependency:
            continue
        if unused_artifact_id in current_project_result.use_dependency or unused_artifact_id in current_project_result.use_transitive_dependency:
            to_version = current_project_result.artifact.split(":")[2]
            return ArtifactExistReport(project_id, unused_artifact_id, from_ts, current_project_result.timestamp, from_version, to_version, i - current_index, "use later")
        to_version = current_project_result.artifact.split(":")[2]
        return ArtifactExistReport(project_id, unused_artifact_id, from_ts, current_project_result.timestamp, from_version, to_version, i - current_index, "removed")
    return ArtifactExistReport(project_id, unused_artifact_id, from_ts, None, from_version, None, None, "stay there")
        

how_long_exist = {}
for project_id, project_reports in rq2_project_result.items():
    result = []
    for project_report in project_reports:
        ts =  compare_result.get_timestamp(project_report.artifact)
        project_report.timestamp = ts
        
    last_index = len(project_reports) - 1
    for index, project_report in enumerate(project_reports):
        for current_unused in project_report.introduce_unused_dependency:
            result.append(find_evolution(index, last_index, project_reports, current_unused, project_id))
        for current_unused in project_report.unused_previously_used:
            result.append(find_evolution(index, last_index, project_reports, current_unused, project_id))
    if len(result) > 0:
        how_long_exist[project_id] = result
            
            
print(how_long_exist)

{'nl.hsac:hsac-fitnesse-email': [<__main__.ArtifactExistReport object at 0x140d8f0b0>, <__main__.ArtifactExistReport object at 0x14300f980>], 'io.github.stefanofornari:vfs2nio': [<__main__.ArtifactExistReport object at 0x140a10080>, <__main__.ArtifactExistReport object at 0x140a10a70>, <__main__.ArtifactExistReport object at 0x140a104a0>, <__main__.ArtifactExistReport object at 0x140a13ce0>, <__main__.ArtifactExistReport object at 0x140a13c80>, <__main__.ArtifactExistReport object at 0x140a10260>, <__main__.ArtifactExistReport object at 0x140a11220>, <__main__.ArtifactExistReport object at 0x140a110d0>], 'com.squareup.pagerduty:pagerduty-incidents': [<__main__.ArtifactExistReport object at 0x140a11010>, <__main__.ArtifactExistReport object at 0x140a11340>, <__main__.ArtifactExistReport object at 0x140a11a60>, <__main__.ArtifactExistReport object at 0x140a10b00>, <__main__.ArtifactExistReport object at 0x140a116a0>, <__main__.ArtifactExistReport object at 0x140a11970>], 'uk.co.rgordon:o

In [144]:
how_long_exist

{'nl.hsac:hsac-fitnesse-email': [<__main__.ArtifactExistReport at 0x140d8f0b0>,
 'io.github.stefanofornari:vfs2nio': [<__main__.ArtifactExistReport at 0x140a10080>,
 'com.squareup.pagerduty:pagerduty-incidents': [<__main__.ArtifactExistReport at 0x140a11010>,
 'uk.co.rgordon:oj-ssh': [<__main__.ArtifactExistReport at 0x140a10e60>,
 'de.vandermeer:ascii-utf-themes': [<__main__.ArtifactExistReport at 0x140a10d40>,
 'io.quarkus.qlue:qlue': [<__main__.ArtifactExistReport at 0x140a11fa0>,
 'com.forgerock.openbanking:openbanking-sdk': [<__main__.ArtifactExistReport at 0x1424591c0>,
 'de.jpdigital:hibernate6-ddl-maven-plugin-core': [<__main__.ArtifactExistReport at 0x140a12990>,
 'io.github.eduramiba:webcam-capture-driver-native': [<__main__.ArtifactExistReport at 0x140a12f60>],
 'com.pastdev:jsch-nio': [<__main__.ArtifactExistReport at 0x140a11e50>,
 'tech.energyit:fast-java-statsd-client': [<__main__.ArtifactExistReport at 0x140a129c0>,
 'io.github.okvee:tscfg-docgen-core': [<__main__.Artif

In [145]:
report_df_list = []

for k, v in how_long_exist.items():
    for artifact_report in v:
        report_df_list.append(artifact_report.convert_to_df())
        
artifact_reports = pd.concat(report_df_list, axis=0, ignore_index=True)
artifact_reports

,project_id,artifact,from_ts,to_ts,last_state,from_version,to_version,version_different,duration
0,nl.hsac:hsac-fitnesse-email,org.fitnesse:fitnesse,1512902820000,1573307166000,use later,0.0.1,0.4.0,4,60404346000
1,nl.hsac:hsac-fitnesse-email,junit:junit,1512902820000,None,stay there,0.0.1,None,None,None
2,io.github.stefanofornari:vfs2nio,junit:junit,1701563362000,None,stay there,0.10.0,None,None,None
3,io.github.stefanofornari:vfs2nio,org.apache.commons:commons-compress,1701563362000,None,stay there,0.10.0,None,None,None
4,io.github.stefanofornari:vfs2nio,org.assertj:assertj-core,1701563362000,None,stay there,0.10.0,None,None,None
...,...,...,...,...,...,...,...,...,...
1570,dev.mccue:guava,dev.mccue:guava-escape,1690819812000,None,stay there,0.0.1,None,None,None
1571,dev.mccue:guava,dev.mccue:guava-collect,1690819812000,None,stay there,0.0.1,None,None,None
1572,dev.mccue:guava,dev.mccue:guava-io,1690819812000,None,stay there,0.0.1,None,None,None
1573,dev.mccue:guava,dev.mccue:guava-primitives,1690819812000,None,stay there,0.0.1,None,None,None


In [146]:
#  t = t.groupby("artifact").agg(
#     introduce_unused_dependency_sum=pd.NamedAgg(column="introduce_unused_dependency", aggfunc="sum"),
#     introduce_unused_dependency_but_import_sum=pd.NamedAgg(column="introduce_unused_dependency_but_import", aggfunc="sum"),
#     average_unused_dependency=pd.NamedAgg(column="unused_dependency", aggfunc="mean"),
#     average_use_transitive_dependency=pd.NamedAgg(column="use_transitive_dependency", aggfunc="mean"),
#     version_count=pd.NamedAgg(column="version", aggfunc="count"),
#     use_transitive_dependency_sum=pd.NamedAgg(column="use_transitive_dependency", aggfunc="sum"),
#     use_dependency_sum=pd.NamedAgg(column="use_dependency", aggfunc="sum"),
# )
 
group_by_state_artifact_report = artifact_reports.groupby("last_state").agg(
    count=pd.NamedAgg(column="artifact", aggfunc="count"),
)

In [147]:
group_by_state_artifact_report

,count
last_state,
removed,153
stay there,1338
use later,84


In [148]:
artifact_reports[artifact_reports["last_state"] == "removed"]

,project_id,artifact,from_ts,to_ts,last_state,from_version,to_version,version_different,duration
13,com.squareup.pagerduty:pagerduty-incidents,com.google.code.gson:gson,1443029646000,1443031154000,removed,1.1.0,1.1.1,1,1508000
62,pl.wavesoftware:eid-exceptions,org.hamcrest:hamcrest-all,1437268232000,1546117762000,removed,0.1.0,2.0.0,5,108849530000
68,pl.wavesoftware:eid-exceptions,org.slf4j:slf4j-simple,1459348094000,1546117762000,removed,1.2.0,2.0.0,1,86769668000
72,com.github.robtimus:obfuscation-json,org.hamcrest:hamcrest-core,1588514318000,1602423878000,removed,1.0,1.0.1,1,13909560000
73,com.github.robtimus:obfuscation-json,org.mockito:mockito-core,1588514318000,1671630437000,removed,1.0,1.2.1,4,83116119000
...,...,...,...,...,...,...,...,...,...
1433,com.github.princesslana:smalld,org.junit.jupiter:junit-jupiter-params,1552775631000,1591213603000,removed,0.0.4,0.2.4,7,38437972000
1469,at.makubi.maven.plugin:avrohugger-maven-plugin,com.julianpeeters:avrohugger-filesorter_2.11,1505657426000,1633187005000,removed,1.4,1.7,3,127529579000
1518,com.github.gquintana.beepbeep:beepbeep,net.java.dev.jna:jna,1464172760000,1547658742000,removed,v0.1.0,0.3.0,2,83485982000
1538,com.datadoghq:java-dogstatsd-client,net.java.dev.jna:jna-platform,1641913091000,1683022993000,removed,4.0.0,2.13.1,3,41109902000


In [149]:
removed_artifact_report = artifact_reports[artifact_reports["last_state"] == "removed"]
removed_artifact_report['duration_day'] = removed_artifact_report['duration']/(1000 * 60 * 60 * 24)
removed_artifact_report = removed_artifact_report[["duration_day", "version_different"]]
removed_artifact_report

/var/folders/qn/b44z9ynj1j1_3c888q5nw9s00000gn/T/ipykernel_41901/1535701394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  removed_artifact_report['duration_day'] = removed_artifact_report['duration']/(1000 * 60 * 60 * 24)


,duration_day,version_different
13,0.017454,1
62,1259.832523,5
68,1004.278565,1
72,160.990278,1
73,961.992118,4
...,...,...
1433,444.883935,7
1469,1476.036794,3
1518,966.27294,2
1538,475.809051,3


In [150]:
stats_table = removed_artifact_report.agg(['min', 'mean', 'median', 'max'])

stats_table_transposed = stats_table.T
stats_table_transposed

,min,mean,median,max
duration_day,0.003403,500.579113,369.707697,2499.004132
version_different,1.000000,6.444444,3.000000,48.000000


In [151]:
artifact_reports[artifact_reports["last_state"] == "use later"]

,project_id,artifact,from_ts,to_ts,last_state,from_version,to_version,version_different,duration
0,nl.hsac:hsac-fitnesse-email,org.fitnesse:fitnesse,1512902820000,1573307166000,use later,0.0.1,0.4.0,4,60404346000
31,com.forgerock.openbanking:openbanking-sdk,io.springfox:springfox-swagger-ui,1510050109000,1542274488000,use later,1.1.0.0,3.0.0.7,34,32224379000
126,org.jamesframework:james-examples,org.jamesframework:james-extensions,1403700571000,1415812996000,use later,0.1,0.2,1,12112425000
144,com.github.josefd8:gingerspec,com.fasterxml.jackson.core:jackson-core,1556775020000,1616665670000,use later,untagged-0df9d744eb48e54583fc,2.2.1-RC1,28,59890650000
145,com.github.josefd8:gingerspec,mysql:mysql-connector-java,1556775020000,1652065910000,use later,untagged-0df9d744eb48e54583fc,2.2.12,41,95290890000
...,...,...,...,...,...,...,...,...,...
1501,de.powerstat.validation:validation,org.apache.logging.log4j:log4j-api,1565456026000,1589279742000,use later,1.0,2.1,1,23823716000
1519,com.github.gquintana.beepbeep:beepbeep,org.slf4j:slf4j-api,1464172760000,1547658742000,use later,v0.1.0,0.3.0,2,83485982000
1540,com.s24:redjob,org.springframework:spring-test,1466495383000,1538896529000,use later,0.0.2,0.5.9,46,72401146000
1541,com.s24:redjob,com.fasterxml.jackson.core:jackson-core,1466495383000,1470670055000,use later,0.0.2,0.0.6,3,4174672000


In [152]:
use_later_artifact_report = artifact_reports[artifact_reports["last_state"] == "use later"]
use_later_artifact_report['duration_day'] = use_later_artifact_report['duration']/(1000 * 60 * 60 * 24)
use_later_artifact_report = use_later_artifact_report[["duration_day", "version_different"]]
use_later_artifact_report

/var/folders/qn/b44z9ynj1j1_3c888q5nw9s00000gn/T/ipykernel_41901/4281174026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  use_later_artifact_report['duration_day'] = use_later_artifact_report['duration']/(1000 * 60 * 60 * 24)


,duration_day,version_different
0,699.124375,4
31,372.96735,34
126,140.190104,1
144,693.178819,28
145,1102.903819,41
...,...,...
1501,275.737454,1
1519,966.27294,2
1540,837.976227,46
1541,48.317963,3


In [153]:
stats_table = use_later_artifact_report.agg(['min', 'mean', 'median', 'max'])

stats_table_transposed = stats_table.T
stats_table_transposed

,min,mean,median,max
duration_day,0.03287,342.065515,222.832992,1936.031111
version_different,1.00000,6.511905,3.000000,46.000000
